In [1]:
import datetime as dt
import panel as pn
import pandas as pd
import holoviews as hv
from holoviews import streams
import hvplot.pandas
import os
import warnings

#warnings.filterwarnings("ignore")
hv.extension("bokeh")
pn.extension("tabulator")

# Read in csv file
#print(os.getcwd())
#data = pd.read_csv("test_files/test_one_step_hydrograph_pentad.csv")
data = pd.read_csv("../../internal_data/hydrograph_pentad_kghm.csv")
print(data)

      Code  pentad   2000   2001   2002  2003   2004   2005   2006   2007  \
0    15102       1   33.7   30.3   30.9  32.1   32.3   32.5    NaN    NaN   
1    15102       2   28.1   24.6   30.5  33.2   32.5   31.1    NaN    NaN   
2    15102       3   27.1   30.4   24.3  33.5   32.3   29.1    NaN    NaN   
3    15102       4   27.0   29.1   24.6  31.6   31.4   29.0    NaN    NaN   
4    15102       5   27.8   27.8   26.6  29.3   30.3   29.9    NaN    NaN   
..     ...     ...    ...    ...    ...   ...    ...    ...    ...    ...   
931  16936      68  208.0  199.0  239.0   NaN  234.0  208.0  182.0  192.0   
932  16936      69  209.0  193.0  241.0   NaN  241.0  183.0  197.0  188.0   
933  16936      70  184.0  196.0  220.0   NaN  219.0  158.0  206.0  186.0   
934  16936      71  200.0  192.0  214.0   NaN  194.0  144.0  194.0  154.0   
935  16936      72  173.0  195.0  178.0   NaN  184.0  145.0  176.0  142.0   

     ...   2015   2016  2017  2018   2019   2020   2021   2022   2023  2024

In [2]:
dataf = data[data['Code']==15194]
print(dataf)

      Code  pentad  2000  2001  2002  2003  2004  2005  2006  2007  ...  2015  \
288  15194       1  2.27  1.73  1.84  2.23  2.11  1.75  1.70  1.99  ...  2.15   
289  15194       2  2.27  1.73  1.89  2.19  2.11  1.75  1.57  1.84  ...  2.15   
290  15194       3  2.27  1.62  2.00  2.19  2.11  1.75  1.44  1.80  ...  2.15   
291  15194       4  1.94  1.50  2.00  2.00  2.11  1.60  1.53  1.80  ...  2.22   
292  15194       5  1.85  1.50  1.96  1.94  2.00  1.58  1.64  1.80  ...  2.26   
..     ...     ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
355  15194      68  1.73  2.02  2.55  2.11  1.94   NaN  2.31  1.93  ...  2.80   
356  15194      69  1.73  2.02  2.55  2.11  1.81   NaN  2.45  1.74  ...  2.74   
357  15194      70  1.73  2.02  2.55  2.11  1.75   NaN  2.87  1.73  ...  2.65   
358  15194      71  1.73  2.02  2.49  2.11  1.75   NaN  2.31  1.73  ...  2.65   
359  15194      72  1.73  1.96  2.23  2.11  1.75   NaN  1.99  1.73  ...  2.65   

     2016  2017  2018  2019

In [3]:
# Show the legend in the hv plot
#p = hv.Curve(dataf, kdims='pentad', vdims=['2000']).opts(legend_position='top_left')

p = hv.Curve(dataf, kdims='pentad', vdims=['2000'], label='2000') *\
hv.Curve(dataf, kdims='pentad', vdims=['2001'], label='2001') *\
hv.Curve(dataf, kdims='pentad', vdims=['2002'], label='2002') *\
hv.Curve(dataf, kdims='pentad', vdims=['2003'], label='2003')
p.opts(width=600, legend_position='top_left')
p


:Overlay
   .Curve.A_2000 :Curve   [pentad]   (2000)
   .Curve.A_2001 :Curve   [pentad]   (2001)
   .Curve.A_2002 :Curve   [pentad]   (2002)
   .Curve.A_2003 :Curve   [pentad]   (2003)

# Experiment with interactive data table and scatter plot


In [16]:
df = data[["Code", "pentad", "2000", "2001"]]

station_widget = pn.widgets.Select(name='Station', options=data['Code'].unique().tolist(), value=15194)

def update_data(value):
    print("Widget\n", value)
    dfu = df[df['Code'] == value]
    return dfu

def update_scatter(value):
    dfp = update_data(value)
    print("Map dfu\n", dfp.head())
    scatter_plot = dfp.hvplot(kind="scatter", x="2000", y="2001", c="pentad").opts(title=f"{value}")
    return scatter_plot

scatter_plot = pn.panel(pn.bind(update_scatter, station_widget))

layout = pn.Column(
    station_widget,
    pn.Row(
        scatter_plot,
    ))

layout.servable()


Widget
 15194
Map dfu
       Code  pentad  2000  2001
288  15194       1  2.27  1.73
289  15194       2  2.27  1.73
290  15194       3  2.27  1.62
291  15194       4  1.94  1.50
292  15194       5  1.85  1.50


BokehModel(combine_events=True, render_bundle={'docs_json': {'a7221d8f-acd4-4078-bf61-0fdcf464cf84': {'version…

In [60]:
import panel as pn
import pandas as pd
import holoviews as hv
from holoviews import streams

pn.extension("tabulator")

df = pd.DataFrame(data={"c": ["a", "a", "b"], "x": [0, 5, 2], "y": [0, 3, 7]})

station = pn.widgets.Select(name='Station', options=df['c'].unique().tolist(), value='a')

def filter_df(df, value):
    return df[df['c'] == value]

scatter = hv.Scatter(filter_df(df, station.value), "x", "y").opts(xlim=(-10, 10),
                                        size=10,
                                        selected=[1],
                                        tools=['tap'])

dff = pn.bind(filter_df, station)

@pn.depends(station.param.value)
def update_table(value):
    ''' link scatter plot selection to table row '''
    print('station_selected', value)
    #dffi = filter_df(df, value)
    tabulator = pn.widgets.Tabulator(dff, selection=[], selectable='checkbox')
    return tabulator

def update_scatter(selection):
    ''' link table row selection to scatter plot '''
    print('row_selected', selection)
    return hv.Scatter(dff, "x", "y").opts(xlim=(-10, 10),
                                         ylim=(-10, 10),
                                         size=10,
                                         selected=selection,
                                         tools=['tap', 'box_select'])

def point_selected(index):
    ''' link scatter plot selection to table row '''
    print('point_selected', index)
    tabulator.selection = index


tabulator = update_table(station.value)
table = pn.panel(tabulator)
plot = hv.DynamicMap(pn.bind(update_scatter, tabulator.param.selection))

sel = streams.Selection1D(source=plot)

sel.param.watch_values(point_selected, 'index')
station.param.watch(update_table, 'value')
# tabulator.param.watch_values(row_selected, 'selection')

pn.Column(station,
          pn.Row(table, plot))#.servable()

station_selected a


TypeError: filter_df() missing 1 required positional argument: 'value'

In [ ]:

df = data[["Code", "2000", "2001"]]
station = pn.widgets.Select(name='Code', options=data['Code'].unique().tolist(), value=15194)
print("station: ", station.value)

@pn.depends(selected_code=station.param.value)
def update_table(selected_code):
    # Print datatypes in dfsel
    dfsel = df[df['Code']==selected_code]
    return dfsel

tabulator = pn.widgets.Tabulator(update_table, selection=[])

@pn.depends(selection=tabulator.param.selection, station=station.param.value)
def update_scatter(selection, station):
    print('station_selected', station)
    ''' link table row selection to scatter plot '''
    print('row_selected', selection)
    dfsel = update_table(station)
    axmin = min(min(dfsel['2000']), min(dfsel['2001']))
    axmax = max(max(dfsel['2000']), max(dfsel['2001']))
    return hv.Scatter(dfsel, "2000", "2001").opts(size=10,
                                         xlim=(axmin, axmax),
                                         ylim=(axmin, axmax),
                                         selected=selection,
                                         tools=['tap', 'box_select'])

#plot = pn.panel(update_scatter)

def point_selected(index):
    ''' link scatter plot selection to table row '''
    print('point_selected', index)
    tabulator.selection = index

plot = hv.DynamicMap(pn.bind(update_scatter, tabulator.param.selection, station))

sel = streams.Selection1D(source=plot)

sel.param.watch_values(point_selected, 'index')

@pn.depends(station.param.value)
def update_data(value):
    dfsel = update_table(value)
    tabulator.value = dfsel
    return dfsel

idf = df.interactive()

#plot.add_periodic_callback(lambda: point_selected(sel.index), 500)

# tabulator.param.watch_values(row_selected, 'selection')

pn.Row(station, idf, tabulator, plot)

station:  15194


AttributeError: 'function' object has no attribute 'owner'

: 